In [1]:
# packages
import os 
import sys
import random
import numpy as np 
from numpy import linalg as LA
from time import process_time

In [2]:
# method to read data from file
# OUTPUT: adjacency matrix
def read(filename):
    PROJECT_PATH = "./DIMACS/"
    path_to_file = os.path.join(PROJECT_PATH, filename)
    
    with open(path_to_file, 'r') as file:
        for line in file:
            if line.startswith('c'):  # graph description
                line.split()[1:]
                #print(*line.split()[1:])
            # first line: p name num_of_vertices num_of_edges
            elif line.startswith('p'):
                p, name, vertices_num, edges_num = line.split()
                print('{0} {1} {2}\n'.format(name, vertices_num, edges_num))
                adj = np.zeros((int(vertices_num),int(vertices_num)))
            elif line.startswith('e'):
                _, v1, v2 = line.split()
                adj[int(v1)-1][int(v2)-1]=1
                adj[int(v2)-1][int(v1)-1]=1
            else:
                continue
                
        #return adjacency matrix        
        return int(vertices_num), adj  

In [13]:
num, adj=read("c1000.9.clq") #Take a look at the graph size/ adjacency

col 1000 450079



COMPLEMENTARY METHODS:

In [14]:
# output the function value of 0.5*x'Ax
def func_val(x, Q):
    val = 0.5*x.transpose().dot(Q).dot(x)
    return val

# check the final result if it is indeed a clique
def check(adj,index):
    for i in index:
        for j in index:
            if i!=j and adj[i][j]!=1:
                return "false"
            
    return "true"

MAIN ALGORITHM:

In [15]:
#Armijo Backtracking
def case_1(vertice_num, x, adj, max_iters, delta, mu, epsilon):
    # Modify the adjacency matrix
    Id = np.identity(vertice_num)
    Q= -(adj+1/2*Id)
    L = np.sqrt(np.trace(Q.dot(Q)))     # L=root(trace(A'A))
    
    for iteration in range(max_iters):
        
##1     # Hessian derived from entropy function
        hessian = np.diag(1/x)

    
##2     # step direction
        Qx = Q.dot(x)
        v_x = -x*(Qx - x.dot(Qx))

    
##3     # step-size
        #--descent inequality        
        vHv = (v_x/x).dot(v_x)
        alpha = 0.95*(2*vHv)/(L*v_x.dot(v_x))
        
##4     # set test point        
        x_hat = x + alpha*v_x

        #check feasibility
        if x_hat.min()<0:
            np.min([x[i]/(-v_x[i]) for i in range(vertice_num) if v_x[i]<0]) # alpha_0= min(x_i/(-v_i) for v<0)            


##5     # sufficient decrease
        f_val = 0.5*x.dot(Qx)
        f_add = mu*vHv
        while func_val(x_hat, Q) > f_val - alpha*f_add:
            alpha = alpha*delta     #shrink step size
            x_hat = x + alpha*v_x     # update test point


##6     # stopping Rule
        if (-func_val(x_hat,Q)+f_val)<(epsilon):
            break

##7     # new state
        old_x = x   
        x = x_hat
        
    return x

INITIALIZE X, RUN ALGORITHM THEN OUTPUT FINAL RESULT

In [16]:
def solve(filename):
    # read the adjacency matrix 
    vertice_num, adj = read(filename)
    print("SOLUTION: ")
    
    # choose starting point at the middle of the simplex
    x=np.array([1/vertice_num for i in range(vertice_num)]) 
    
    # RUN ALGORITHM 
    # max 10^6 iteration, shrinking factor delta=0.5, mu=0.001, epsilon=10^(-9) chosen
    start = process_time()
    sol = case_1(vertice_num, x, adj, 1000000, 0.7, 0.1, 0.000000001)
    end = process_time()
    
    # retrieve the index of chosen vertices
    ans = np.array([i for i in sol if i>0.1/len(sol)])
    index = np.array([i for i in range(len(sol)) if sol[i]>0.1/len(sol)])
    
    print(' '.join(str(x+1) for x in index))
    print("Maximum clique size: {}\n".format(len(ans)))
    print("Clique: {}".format(check(adj,index)))
    print('CPU = {}'.format(np.mean(end-start)))
    
    return sol

SOLVE HERE

In [119]:
solution = solve("keller4.clq")  # epsilon = 10^(-9)

edge 171 9435

SOLUTION: 
3 14 20 73 79 103 113
Maximum clique size: 7

Clique: true
CPU = 0.125


In [9]:
# RUN WITH MULTIPLE STARTING POINTS 

def multi_solve(filename, trials):
    # input the adjacency matrix
    vertice_num, adj = read(filename)

    
    #running trials of different starting x
    print('{} trials:'.format(trials)) 
    solutions = []; time = []
    Id = np.identity(vertice_num)
    A= -(adj+1/2*Id)
    
    for i in range(trials):
        print("process: {}".format(i+1))
        # choose the starting point at random
        x=np.random.dirichlet(np.ones(vertice_num)).ravel()
 
        # run algorithm
        start = process_time()
        sol = case_1(vertice_num, x, adj, 1000000, 0.7, 0.01, 0.000000001)
        end = process_time()
    
        # retrieve the index of the chosen vertices
        ans = np.array([i for i in sol if i>0.1/len(sol)]) 
        index = np.array([i for i in range(len(sol)) if sol[i]>0.1/len(sol)])
        
        if check(adj,index):
            solutions.append(len(ans))
            time.append(end-start)
    
    return np.array(solutions), np.array(time)

In [67]:
file = "brock400_2.clq"

In [69]:
solutions,times = multi_solve(file, 50)

edge 400 59786

50 trials:
process: 1
process: 2
process: 3
process: 4
process: 5
process: 6
process: 7
process: 8
process: 9
process: 10
process: 11
process: 12
process: 13
process: 14
process: 15
process: 16
process: 17
process: 18
process: 19
process: 20
process: 21
process: 22
process: 23
process: 24
process: 25
process: 26
process: 27
process: 28
process: 29
process: 30
process: 31
process: 32
process: 33
process: 34
process: 35
process: 36
process: 37
process: 38
process: 39
process: 40
process: 41
process: 42
process: 43
process: 44
process: 45
process: 46
process: 47
process: 48
process: 49
process: 50


In [57]:
print('{} return true clique'.format(len(solutions)))
print('max = {}'.format(np.amax(solutions)))
print('mean = {}'.format(np.mean(solutions)))
print('std = {}'.format(np.std(solutions)))
print('CPU = {}'.format(np.mean(times)))

100 return true clique
max = 15
mean = 11.96
std = 1.0575443253121828
CPU = 0.30296875


In [58]:
result.append([file,len(solutions),np.amax(solutions),np.mean(solutions),np.std(solutions),np.mean(times)])

In [59]:
result

[['c125.9.clq', 100, 33, 28.46, 1.7459667808981933, 0.0840625],
 ['c250.9.clq', 100, 40, 35.11, 1.7938506069347022, 0.30171875],
 ['c500.9.clq', 100, 48, 42.64, 2.0075856146127364, 9.675625],
 ['DSJC500_5.clq', 50, 11, 9.38, 0.7453858061433689, 8.7365625],
 ['DSJC1000_5.clq', 50, 13, 10.64, 0.8662563131083085, 51.8778125],
 ['brock200_2.clq', 100, 10, 7.82, 0.7665507158694722, 0.22703125],
 ['brock200_4.clq', 100, 15, 11.96, 1.0575443253121828, 0.30296875]]